In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
import torch
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from collections import Counter
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report
from Scripts.Configs.ConfigClass import Config
from torch_geometric.data import Data
from torch_geometric.utils import to_networkx
import networkx as nx
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data.dataset import random_split, T_co
from torch_geometric.data.lightning import LightningDataset
import pdb
import lightning as L
import time
from Scripts.DataManager.GraphConstructor.CoOccurrenceGraphConstructor import CoOccurrenceGraphConstructor
from Scripts.DataManager.GraphLoader.GLabeledGraphLoader import GLabeledGraphLoader

stop_words = set(stopwords.words('english'))
stemmer = SnowballStemmer('english')
config = Config(r'C:\Users\fardin\Projects\ColorIntelligence\Scripts\Configs\Config.json')

In [6]:
train_df = pd.read_csv(r'C:\Users\fardin\Projects\ColorIntelligence\data\Amazon-Review\train_sm.csv')
test_df = pd.read_csv(r'C:\Users\fardin\Projects\ColorIntelligence\data\Amazon-Review\test_sm.csv')
train_df.columns = ['Polarity', 'Title', 'Review']
test_df.columns = ['Polarity', 'Title', 'Review']
train_df = train_df[['Polarity', 'Review']]
test_df = test_df[['Polarity', 'Review']]

In [8]:
start_time = time.time()
graph_const = CoOccurrenceGraphConstructor(train_df['Review'], 'AmazonReview', config, lazy_construction=True,  load_preprocessed_data=True, naming_prepend='graph')
print(f'execution time in second: {time.time() - start_time}')
# graph_const = CoOccurrenceGraphConstructor(train_df['Review'][:10], 'AmazonReview', config, lazy_construction=False, naming_prepend='graph', load_preprocessed_data=False)
# graph = graph_const.to_graph(train_df['Review'][0])
# graph_const.draw_graph(0)

execution time in second: 0.009000062942504883


In [9]:
labels = torch.tensor(test_df['Polarity'].apply(lambda p: 0 if p==1 else 1), dtype=torch.float32).view(-1,1)
graph_loader = GLabeledGraphLoader(graph_const, labels[:10],2, 'cpu', val_size=0.2)

In [10]:
from typing import Any
import torch
from pytorch_lightning.utilities.types import OptimizerLRScheduler, STEP_OUTPUT
import lightning as L
from torch_geometric.nn import  GCNConv
from torch import nn

In [11]:
train_data_loader = graph_loader.get_train_data()
X, y = next(iter(train_data_loader))

In [12]:
from Scripts.Models.ClassifierModels.GraphAutoEncoderModel1 import GraphAutoEncoderModel1
autoencoder_model = GraphAutoEncoderModel1(300, 1)
autoencoder_model(X)

tensor([[59.3724],
        [43.7305]], grad_fn=<AddmmBackward0>)

In [13]:
from Scripts.Models.LightningModels.LightningClassifier1 import BinaryLightningModel

autoencoder_model = GraphAutoEncoderModel1(300, 1)
lightning_model = BinaryLightningModel(autoencoder_model,
                                 torch.optim.Adam(autoencoder_model.parameters(), lr=0.001, weight_decay=0.005), nn.BCEWithLogitsLoss())

In [14]:
trainer = L.Trainer(max_epochs=10, accelerator='gpu', devices=1, num_sanity_val_steps=0)
trainer.fit(lightning_model, graph_loader.get_train_data(), graph_loader.get_val_data())

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type                   | Params
-----------------------------------------------------
0 | model     | GraphAutoEncoderModel1 | 245 K 
1 | loss_func | BCEWithLogitsLoss      | 0     
2 | train_acc | BinaryAccuracy         | 0     
3 | val_acc   | BinaryAccuracy         | 0     
4 | test_acc  | BinaryAccuracy         | 0     
-----------------------------------------------------
245 K     Trainable params
0         Non-trai

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

C:\Users\fardin\AppData\Local\Programs\Python\Python310\lib\site-packages\lightning\pytorch\utilities\data.py:77: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 144. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.


Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.


In [23]:
X, y = next(iter(graph_loader.get_test_data()))
print(y)
print(lightning_model(X))

tensor([[1.],
        [1.]])
tensor([[-26.4988],
        [ -5.2990]], grad_fn=<AddmmBackward0>)


tensor([[-66.1845],
        [ 17.6578]], grad_fn=<AddmmBackward0>)

In [29]:
train_data = graph_loader.get_train_data()

In [54]:
X, y = next(iter(train_data))
module_0 = GCNConv(300, 256)
output = module_0(X.x, X.edge_index, X.edge_attr)

In [57]:
output2 = [output[:X[0].x.shape[0]], output[X[0].x.shape[0]:X[1].x.shape[0]], output[X[1].x.shape[0]:X[2].x.shape[0]]]

In [140]:
import torch_geometric

In [142]:
print(X is torch_geometric.data.batch.Batch)

False


In [70]:
gcn_conv = GCNConv(256, 64)

In [95]:
# s = gcn_conv(output2[0], X[0].edge_index)
# adj = torch.sparse_coo_tensor(X[0].edge_index, X[0].edge_attr).to_dense()
# output3 = dense_diff_pool(output2[0], adj, s=s)

In [63]:
from flags import Flags
class SampleGraph(Flags):
    val1 = 1
    val2 = 2
    val3 = 4
    val4 = 8

In [64]:
sample: SampleGraph = SampleGraph.val4 | SampleGraph.val3

In [66]:
SampleGraph.val4 == sample

False